# Neptune + Keras Tuner

## Before you start

### Install dependencies

In [ ]:
! pip install --quiet keras-tuner==1.0.2 tensorflow==2.4.1 plotly==4.14.3 neptune-client==0.4.133 neptune-contrib[monitoring]==0.25.0

In [ ]:
! pip install --quiet keras-tuner tensorflow plotly neptune-client neptune-contrib[monitoring] --upgrade

### Import libraries and prepare dataset

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

from kerastuner.tuners import  BayesianOptimization

(x, y), (val_x, val_y) = keras.datasets.mnist.load_data()
x = x.astype('float32') / 255.
val_x = val_x.astype('float32') / 255.

x = x[:10000]
y = y[:10000]

### Create a model building function

In [ ]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=(28, 28)))
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i), 32, 512, 32),
                               activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    return model

### Initialize Neptune

In [ ]:
import neptune

neptune.init(api_token='ANONYMOUS', project_qualified_name='shared/keras-tuner-integration')

Neptune gives you an option of logging data under a public folder as an anonymous user. This is great when you are just trying out the application and don't have a Neptune account yet.  

---

If you already have a [Neptune account](https://neptune.ai/register), you can create your own experiment and start logging to it using your personal API token. Pass your `username` to the `project_qualified_name` argument of the `neptune.init()` method: `project_qualified_name='YOUR_USERNAME/YOUR_PROJECT_NAME`. If you don't have a project yet, keep `/sandbox` at the end. The `sandbox` project is automatically created for you.

## Quickstart

### Step 1: Create an Experiment

This creates an experiment in Neptune.

Once you have a live experiment you can log things to it. 

In [4]:
neptune.create_experiment('bayesian-sweep')

Info (NVML): NVML Shared Library Not Found. GPU usage metrics may not be reported. For more information, see https://docs.neptune.ai/logging-and-managing-experiment-results/logging-experiment-data.html#hardware-consumption 


https://ui.neptune.ai/shared/keras-tuner-integration/e/KER-19


Experiment(KER-19)

Click on the link above to open this experiment in Neptune.

For now it is empty but keep the tab with experiment open to see what happens next. 

### Step 2: Pass Neptune Logger to Tuner

In [5]:
import neptunecontrib_kerastuner as npt_utils 

tuner =  BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    num_initial_points=3,
    executions_per_trial=3,
    project_name='bayesian-sweep',
    logger=npt_utils.NeptuneLogger())

INFO:tensorflow:Reloading Oracle from existing project ./bayesian-sweep/oracle.json
INFO:tensorflow:Reloading Tuner from ./bayesian-sweep/tuner0.json


### Step 3: Run the search and monitor it in Neptune 

You can view the logging live in the Neptune tab once you run the below cell

In [6]:
tuner.search(x=x,
             y=y,
             epochs=5,
             validation_data=(val_x, val_y))

Trial 5 Complete [00h 00m 12s]
val_accuracy: 0.9132000009218851

Best val_accuracy So Far: 0.9533666769663492
Total elapsed time: 00h 02m 23s
INFO:tensorflow:Oracle triggered exit


### Step 4: Log additional sweep information after the sweep

Log information like best parameters, best score, search space, and project dir to Neptune. 

In [7]:
npt_utils.log_tuner_info(tuner)

### Step 5: Stop logging

When you track experiments with Neptune in Jupyter notebooks you need to explicitly stop the experiment by running ```neptune.stop()```.

If you are running Neptune in regular ```.py``` scripts it will stop automatically when your code stops running.

In [8]:
# tests
exp = neptune.get_experiment()

In [9]:
neptune.stop()

In [11]:
# tests
all_logs = exp.get_logs()

## check logs
correct_logs = ['val_accuracy', 'hyperparameters/values', 'val_loss', 'loss', 'accuracy', 'run_score', 'hyperparameters/space', 'best_score']

assert set(all_logs.keys()) == set(correct_logs), 'Expected: {}. Actual: {}'.format(set(correct_logs), set(all_logs.keys()))

all_properties = exp.get_properties()

## check logs
correct_properties = ['objective/name', 'best_trial_id', 'best_parameters', 'tuner_id', 'objective/direction']

assert set(all_properties.keys()) == set(correct_properties), 'Expected: {}. Actual: {}'.format(set(correct_properties), set(all_properties.keys()))

## Explore results in the Neptune UI